In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def read_list(listfile):
    '''
    reads a newline separated list like created by save_list
    args:
        listfile: path to a textfile containing a newline separated list
    returns:
        a list of strings saved in the list file
    '''
    return open(listfile, 'r').read().strip('\n').split('\n')

In [3]:
clusts = read_list('../../data/ref/cluster_list.txt')

In [4]:
data_path = '../../data/processed/gex_percent_expressed/{}/{}'

In [5]:
donors = !ls ../../data/processed/gex_percent_expressed/human/ | grep M

In [6]:
donors = [donor.split('_donor_')[0] for donor in donors]

In [7]:
species = ['human', 'macaque', 'marmoset', 'mouse']

In [8]:
counts = '../../data/processed/gex_percent_expressed/{}/all_counts_per_celltype.tsv'
accessed = '../../data/processed/gex_percent_expressed/{}/all_donors_count_each_clust.tsv'
percent = '../../data/processed/gex_percent_expressed/{}/all_percent_aggregate.tsv'
for specie in species:
    all_counts_cells = pd.read_csv(counts.format( specie), sep='\t').set_index('Var1').reindex(clusts, fill_value=0)
    all_counts_expressed = pd.read_csv(accessed.format( specie),
            sep='\t').reindex(columns = clusts, fill_value=0)
    percent_all = all_counts_expressed/all_counts_cells.values.T
    # n_donors_above_threshold.to_csv('../../data/processed/gex_percent_expressed/{}/all_donors_above_thresh.tsv'.format(specie), sep='\t')
    percent_all.to_csv(percent.format(specie), sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: '../../data/processed/gex_percent_expressed/human/all_counts_per_celltype.tsv'

In [ ]:
for specie in species:
    tmp = data_path.format(specie, '')
    dat = data_path.format(specie, '{}')
    print(specie)
    donors = !ls $tmp | grep -v all
    donors = [donor.split('_donor_')[0] for donor in donors]
    donors = np.unique(donors)    
    min_donors_percent = .5
    min_donors_covered = len(donors) * min_donors_percent
    min_expression_percent = .15
    all_counts_cells = []
    all_counts_expressed = []
    n_donors_above_threshold = []
    first = True
    for donor in donors:
        cell_counts = pd.read_csv(
                dat.format('{}_donor_count_each_clust.tsv'.format(donor)),
                sep='\t').set_index('Var1').reindex(clusts, fill_value=0)
        n_expressed = pd.read_csv(
                dat.format('{}_donor_n_expressed_each_clust.tsv'.format(donor)),
                sep='\t').reindex(columns = clusts, fill_value=0).dropna()
        counts_use = cell_counts.loc[n_expressed.columns.values]
        percent_expressed = n_expressed/counts_use.values.T
        # if specie == 'human':
            # print(donor)
            # print(percent_expressed.max())
            
        thresholded = (percent_expressed > .15).astype(int)
        if first:
            all_counts_cells = cell_counts
            all_counts_expressed = n_expressed
            n_donors_above_threshold = thresholded
            first = False
        else:
            all_counts_cells = all_counts_cells + cell_counts
            all_counts_expressed = all_counts_expressed + n_expressed
            n_donors_above_threshold = n_donors_above_threshold + thresholded
    
    if specie == 'human':
        all_counts_cells = pd.read_csv('../../data/processed/gex_percent_expressed/human_tmp/all_donor_count_each_clust.tsv', sep='\t').set_index('Var1').reindex(clusts, fill_value=0)
        all_counts_expressed = pd.read_csv(
                '../../data/processed/gex_percent_expressed/human_tmp/all_expression_counts.tsv',
                sep=' ').reindex(columns = clusts, fill_value=0).dropna()
    percent_all = all_counts_expressed/all_counts_cells.values.T
    n_donors_above_threshold.to_csv('../../data/processed/gex_percent_expressed/{}/all_donors_above_thresh.tsv'.format(specie), sep='\t')
    percent_all.to_csv('../../data/processed/gex_percent_expressed/{}/all_percent_aggregate.tsv'.format(specie), sep='\t')

In [9]:
all_counts_expressed.head()

NameError: name 'all_counts_expressed' is not defined